# Pre stuff
Importing libraries, loading and visualising data.

In [1]:
import mne

from mne.decoding import Vectorizer

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.model_selection import train_test_split

In [21]:
# Folder & files containing the data:
data_path = 'data/'
data_file = '817_1_PDDys_ODDBALL_Clean_curated'

filename = data_path + data_file

# We read the EEG epochs:
epochs = mne.read_epochs(filename + '.fif')

num_filter = 10

This filename (data/817_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
Reading data/817_1_PDDys_ODDBALL_Clean_curated.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
189 matching events found
No baseline correction applied


<ipython-input-21-fe22cab9f260>:8: RuntimeWarning: This filename (data/817_1_PDDys_ODDBALL_Clean_curated.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(filename + '.fif')


189 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated


In [3]:
epochs.info

<Info | 18 non-empty fields
    bads : list | 0 items
    ch_names : list | Fp1, Fz, F3, F7, FC5, FC1, C3, T7, CP5, ...
    chs : list | 60 items (EEG: 60)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : Transform | 3 items
    dig : list | 60 items
    events : list | 0 items
    file_id : dict | 4 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 250.0 Hz
    meas_id : dict | 4 items
    nchan : int | 60
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 500.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    experimenter : NoneType
    gantry_angle : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_date : NoneType
    proj_id : NoneType
    proj_name : NoneType
    subject_info : NoneType
    xplotter_layout : NoneTyp

In [4]:
# The data can be accessed via:
epochs._data

array([[[ 3.52394723e-06,  2.06192612e-06,  4.46908902e-06, ...,
          2.96665620e-06,  2.16401624e-06,  1.87072610e-06],
        [ 4.50709437e-06,  4.35066937e-06,  6.42107294e-06, ...,
          3.37508463e-06,  3.61751079e-06,  7.65960632e-07],
        [-1.29490099e-05,  1.14769172e-06,  7.53064344e-06, ...,
          9.18080709e-06,  9.31064699e-06,  7.09726427e-06],
        ...,
        [ 4.06841724e-07,  2.90893554e-06,  4.71284770e-06, ...,
          9.70285413e-06,  6.65089987e-06,  5.53862809e-06],
        [-3.15314721e-06, -3.34791898e-06,  2.23534035e-06, ...,
          6.38878630e-06,  3.25191187e-06, -5.72537182e-07],
        [ 5.18393038e-06,  3.53263568e-06,  3.91299509e-06, ...,
         -8.49513529e-06, -8.71153162e-06, -9.73261926e-06]],

       [[ 1.03230610e-05,  6.71308611e-06,  4.06666993e-06, ...,
          1.60698795e-05,  9.03341291e-06,  8.83130644e-06],
        [ 1.22205867e-05,  1.32242498e-05,  1.39072570e-05, ...,
          2.48241805e-06,  8.66339622e

In [5]:
epochs.plot

<bound method BaseEpochs.plot of <EpochsFIF  |   189 events (all good), -0.1 - 0.5 sec, baseline off, ~26.2 MB, data loaded,
 'Novel': 30
 'Standard': 130
 'Target': 29>>

In [7]:
# save original data multiple times to apply different filters to it without altering the original data
multi_epochs = [epochs] * num_filter
print(len(multi_epochs))

10


In [8]:
events = epochs.events
print('Events = ')
print(events)

Events = 
[[  1001      0    201]
 [  2680      0    201]
 [  4019      0    201]
 [  6379      0    201]
 [  8291      0    201]
 [ 10297      0    200]
 [ 12592      0    201]
 [ 14380      0    201]
 [ 16383      0    201]
 [ 18261      0    201]
 [ 20287      0    200]
 [ 22321      0    201]
 [ 24411      0    201]
 [ 26467      0    200]
 [ 28488      0    201]
 [ 30443      0    201]
 [ 32442      0    201]
 [ 34420      0    200]
 [ 36299      0    201]
 [ 38318      0    201]
 [ 40502      0    201]
 [ 44480      0    201]
 [ 46321      0    201]
 [ 50342      0    201]
 [ 52490      0    201]
 [ 56344      0    201]
 [ 58291      0    201]
 [ 60400      0    200]
 [ 62414      0    201]
 [ 64419      0    201]
 [ 68433      0    201]
 [ 72295      0    201]
 [ 74403      0    201]
 [ 76308      0    201]
 [ 78312      0    200]
 [ 80345      0    201]
 [ 82478      0    201]
 [ 86437      0    201]
 [ 88335      0    200]
 [ 90332      0    201]
 [ 92268      0    201]
 [ 964

In [11]:
epochs.event_id

{'Standard': 201, 'Target': 200, 'Novel': 202}

# Preprocessing

## Analysing the data (Max)

## Filtering the data (Julie and Georgi)

# Applying Machine Learning

In [52]:
# get the data
all_data = [] # create empty list
for epochs_i in multi_epochs:
    all_data.append(epochs_i._data) # append new data object to list

# get the labels
labels = epochs.events[:,-1]

print()
print('Length of data:')
print(len(all_data))
#print('Data:')
#print(all_data)
print('Length of labels:')
print(len(labels))


Length of data:
10
Length of labels:
189


In [42]:
# create pipeline to execute multiple steps in a row
clf = [make_pipeline(Vectorizer(), StandardScaler(),
                    svm.SVC(kernel='linear',C=1))] * num_filter

In [55]:
for counter, clf_i in enumerate(clf):
    print('Step', counter+1, '/', len(all_data))
    data_train, data_test, labels_train, labels_test = train_test_split(all_data[counter], labels, test_size=0.4, random_state=0)
    clf_i.fit(data_train, labels_train) # apply pipelined steps
    clf_i.score(data_train,labels_train)

Step 1 / 10
Step 2 / 10
Step 3 / 10
Step 4 / 10
Step 5 / 10
Step 6 / 10
Step 7 / 10
Step 8 / 10
Step 9 / 10
Step 10 / 10
